## 39 Renderizando sprites animados

As imagens usadas neste notebook foram obtidas nos sites:

https://www.pngfuel.com

https://www.kindpng.com

https://opengameart.org

https://www.hiclipart.com

https://www.pngwing.com/

In [5]:
import glm
import time
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v3 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v4 import ModelRenderer
from cg.models.SquareMesh_v3 import SquareMesh
from cg.utils.Texture import Texture

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        self.cameraPos   = glm.vec3(0.0, 0.0, 1.2)
        self.cameraFront = glm.vec3(0.0, 0.0, 0.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
    
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
        # define uma malha quadrada
        vertex_position = np.array([
            -0.50, -0.50, 0.0, 1.0, # Triângulo 1
             0.50, -0.50, 0.0, 1.0,
            -0.50,  0.50, 0.0, 1.0,
             0.50, -0.50, 0.0, 1.0, # Triângulo 2
             0.50,  0.50, 0.0, 1.0,
            -0.50,  0.50, 0.0, 1.0],
            dtype=np.float32)
        
        # define as coordendas de texturas para renderizá-la por completo
        self.texCoord = np.array([
             0.00,  1.00, # Triângulo 1
             1.00,  1.00,
             0.00,  0.00,
             1.00,  1.00, # Triângulo 2
             1.00,  0.00,
             0.00,  0.00],
            dtype=np.float32)
        
        # DADOS DO SPRITE DE MOEDA ====================================================
        
        # calcula o tamanho de uma parte/quadro/frame da textura
        coin_frame_size = np.array([1.0 / 8, 1.0 / 4], dtype=np.float32)
        self.coinTexCoordFirstFrame = self.texCoord.reshape((6, 2)) * coin_frame_size
        
        # calcula o offset dos outros frames em relação ao primeiro frame
        x = np.linspace(0, 1, 9)
        y = np.linspace(0, 1, 5)
        self.coinFrameOffset_x, self.coinFrameOffset_y = np.meshgrid(x[:-1], y[:-1])
        
        self.coinFrameOffset_x = self.coinFrameOffset_x.reshape((32)).astype(np.float32)
        self.coinFrameOffset_y = self.coinFrameOffset_y.reshape((32)).astype(np.float32)
        
        mask = np.ones(32, dtype=bool)
        mask[[0, 11, 22, 30, 31]] = False
        self.coinFrameSeq = np.arange(0, 32)[mask]
        
        self.coinTexCoordCurrentFrame = self.coinTexCoordFirstFrame
        self.coinCurrentFrameIndex = 0
  

        # DADOS DO SPRITE DO MEGAMAN ====================================================
    
        # calcula o tamanho de uma parte/quadro/frame da textura
        megaman_frame_size = np.array([1.0 / 5, 1.0 / 2], dtype=np.float32)
        self.megamanTexCoordFirstFrame = self.texCoord.reshape((6, 2)) * megaman_frame_size
        
        # calcula o offset dos outros frames em relação ao primeiro frame
        x = np.linspace(0, 1, 6)
        y = np.linspace(0, 1, 3)
        self.megamanFrameOffset_x, self.megamanFrameOffset_y = np.meshgrid(x[:-1], y[:-1])
        
        self.megamanFrameOffset_x = self.megamanFrameOffset_x.reshape((10)).astype(np.float32)
        self.megamanFrameOffset_y = self.megamanFrameOffset_y.reshape((10)).astype(np.float32)
        
        # define a sequencia da animação
        self.megamanFrameSeq = np.arange(0, 10)
        self.megamanTexCoordCurrentFrame = self.megamanTexCoordFirstFrame
        self.megamanCurrentFrameIndex = 0
        
        
        # DADOS DO SPRITE DA EXPLOSÃO ====================================================
        
        # calcula o tamanho de uma parte/quadro/frame da textura
        explosion_frame_size = np.array([1.0 / 8, 1.0 / 6], dtype=np.float32)
        self.explosionTexCoordFirstFrame = self.texCoord.reshape((6, 2)) * explosion_frame_size
        
        # calcula o offset dos outros frames em relação ao primeiro frame
        x = np.linspace(0, 1, 9)
        y = np.linspace(0, 1, 7)
        self.explosionFrameOffset_x, self.explosionFrameOffset_y = np.meshgrid(x[:-1], y[:-1])
        
        self.explosionFrameOffset_x = self.explosionFrameOffset_x.reshape((48)).astype(np.float32)
        self.explosionFrameOffset_y = self.explosionFrameOffset_y.reshape((48)).astype(np.float32)
        
        # define a sequencia da animação
        self.explosionFrameSeq = np.arange(0, 48)
        self.explosionTexCoordCurrentFrame = self.explosionTexCoordFirstFrame
        self.explosionCurrentFrameIndex = 0
        
        
        # DADOS DO SPRITE DE FOGO ====================================================
        
        # calcula o tamanho de uma parte/quadro/frame da textura
        fire_frame_size = np.array([1.0 / 10, 1.0 / 6], dtype=np.float32)
        self.fireTexCoordFirstFrame = (self.texCoord.reshape((6, 2)) * fire_frame_size)
        
        # calcula o offset dos outros frames em relação ao primeiro frame
        x = np.linspace(0, 1, 11)
        y = np.linspace(0, 1, 7)
        self.fireFrameOffset_x, self.fireFrameOffset_y = np.meshgrid(x[:-1], y[:-1])
        
        self.fireFrameOffset_x = self.fireFrameOffset_x.reshape((60)).astype(np.float32)
        self.fireFrameOffset_y = self.fireFrameOffset_y.reshape((60)).astype(np.float32)
        
        # define a sequencia da animação
        self.fireFrameSeq = np.arange(0, 60)
        self.fireTexCoordCurrentFrame = self.fireTexCoordFirstFrame
        self.fireCurrentFrameIndex = 0
        
        # ===========================================================================
        
        # carrega e cria as texturas
        self.texture_1 = Texture('./cg/images/textures/coins.jpg')
        self.texture_2 = Texture('./cg/images/textures/megaman_kindpng.png')
        self.texture_3 = Texture('./cg/images/textures/explosion_pngfuel.com.png')
        self.texture_4 = Texture('./cg/images/textures/fire_opengameart.org.png')
        
        # cria o objeto responsável por carregar os dados para a GPU e renderizá-los
        self.squareRenderer = ModelRenderer(vertex_position, vertex_tex=self.texCoord)
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()

        # recupera o endereço da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        tex_coord_loc = self.shaderProgram.getVertexTextureCoordLoc()
        
        # configura os dados do modelo para serem os dados de entrada do shader program
        self.squareRenderer.setVertexPositionLoc(position_loc)
        self.squareRenderer.setVertexTextureCoordLoc(tex_coord_loc)
        
        # configura a cor de background
        gl.glClearColor(0.9, 0.9, 0.9, 1)
        
        # armazena o momento que o programa começou
        self.startTime = time.time()
        
        # habilita teste de profundidade e culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
        #configura função de mistura de cores
        gl.glBlendFunc(gl.GL_SRC_ALPHA, gl.GL_ONE_MINUS_SRC_ALPHA)
        
    def paintGL(self):
            
        # limpa o background com a cor especificada e o buffer de profundidade
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # atualiza os frames
        self.updateFrames()
        
        # renderiza as animações
        self.renderMegaman()
        self.renderCoin()
        self.renderExplosion()
        self.renderFire()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
    
    def updateFrames(self):
        
        # calcula o tempo de execução do programa
        self.currentTime = time.time()
        time_difference = self.currentTime - self.startTime
        
        if(time_difference > 0.05):
            self.startTime = self.currentTime
            time_difference = 0
            
            # calcula o index do próximo frame
            self.coinCurrentFrameIndex += 1
            self.megamanCurrentFrameIndex += 1
            self.explosionCurrentFrameIndex += 1
            self.fireCurrentFrameIndex += 1
            
            if(self.coinCurrentFrameIndex >= self.coinFrameSeq.size):
                self.coinCurrentFrameIndex = 0
            
            if(self.megamanCurrentFrameIndex >= self.megamanFrameSeq.size):
                self.megamanCurrentFrameIndex = 0
            
            if(self.explosionCurrentFrameIndex >= self.explosionFrameSeq.size):
                self.explosionCurrentFrameIndex = 0
            
            if(self.fireCurrentFrameIndex >= self.fireFrameSeq.size):
                self.fireCurrentFrameIndex = 0
            
            # atualiza as coordenadas de textura dos frames
            offset_x = self.coinFrameOffset_x[self.coinFrameSeq[self.coinCurrentFrameIndex]]
            offset_y = self.coinFrameOffset_y[self.coinFrameSeq[self.coinCurrentFrameIndex]]
            self.coinTexCoordCurrentFrame = self.coinTexCoordFirstFrame + np.array([offset_x, offset_y])
            
            offset_x = self.megamanFrameOffset_x[self.megamanFrameSeq[self.megamanCurrentFrameIndex]]
            offset_y = self.megamanFrameOffset_y[self.megamanFrameSeq[self.megamanCurrentFrameIndex]]
            self.megamanTexCoordCurrentFrame = self.megamanTexCoordFirstFrame + np.array([offset_x, offset_y])
            
            offset_x = self.explosionFrameOffset_x[self.explosionFrameSeq[self.explosionCurrentFrameIndex]]
            offset_y = self.explosionFrameOffset_y[self.explosionFrameSeq[self.explosionCurrentFrameIndex]]
            self.explosionTexCoordCurrentFrame = self.explosionTexCoordFirstFrame + np.array([offset_x, offset_y])
            
            offset_x = self.fireFrameOffset_x[self.fireFrameSeq[self.fireCurrentFrameIndex]]
            offset_y = self.fireFrameOffset_y[self.fireFrameSeq[self.fireCurrentFrameIndex]]
            self.fireTexCoordCurrentFrame = self.fireTexCoordFirstFrame + np.array([offset_x, offset_y])
            
    def renderMegaman(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(-0.7, 1.05, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.shaderProgram.bindTexture2D(self.texture_2.getTextureID())

        # atualiza as coordenadas de textura para renderiza a textura toda
        self.squareRenderer.updateVertexTextureCoord(self.texCoord)
        
        # renderiza o quadrado usando toda a textura
        self.squareRenderer.render()
        
        # habilita o blending
        gl.glEnable(gl.GL_BLEND)
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(0.7, 1.05, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # atualiza a GPU com as coordenadas de textura do frame
        self.squareRenderer.updateVertexTextureCoord(self.megamanTexCoordCurrentFrame)
        
        # renderiza o quadrado com parte da textura
        self.squareRenderer.render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.shaderProgram.releaseTexture2D()
        
        # desabilita o blending
        gl.glDisable(gl.GL_BLEND)
    
    def renderCoin(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(-0.7, 0.0, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.shaderProgram.bindTexture2D(self.texture_1.getTextureID())
        
        # atualiza as coordenadas de textura para renderiza a textura toda
        self.squareRenderer.updateVertexTextureCoord(self.texCoord)
        
        # renderiza o quadrado usando toda a textura
        self.squareRenderer.render()

        # habilita o blending
        gl.glEnable(gl.GL_BLEND)
    
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(0.7, 0.0, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # atualiza a GPU com as coordenadas de textura do frame
        self.squareRenderer.updateVertexTextureCoord(self.coinTexCoordCurrentFrame)
        
        # renderiza o quadrado com parte da textura
        self.squareRenderer.render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.shaderProgram.releaseTexture2D()
        
        # desabilita o blending
        gl.glDisable(gl.GL_BLEND)
    
    def renderExplosion(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(-1.8, -1.05, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.shaderProgram.bindTexture2D(self.texture_3.getTextureID())
        
        # atualiza as coordenadas de textura para renderiza a textura toda
        self.squareRenderer.updateVertexTextureCoord(self.texCoord)
        
        # renderiza o quadrado usando toda a textura
        self.squareRenderer.render()
    
        # habilita o blending
        gl.glEnable(gl.GL_BLEND)
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(-0.7, -1.05, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # atualiza a GPU com as coordenadas de textura do frame
        self.squareRenderer.updateVertexTextureCoord(self.explosionTexCoordCurrentFrame)
        
        # renderiza o quadrado com parte da textura
        self.squareRenderer.render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.shaderProgram.releaseTexture2D()
        
        # desabilita o blending
        gl.glDisable(gl.GL_BLEND)
    
    def renderFire(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(0.7, -1.05, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.shaderProgram.bindTexture2D(self.texture_4.getTextureID())
        
        # atualiza as coordenadas de textura para renderiza a textura toda
        self.squareRenderer.updateVertexTextureCoord(self.texCoord)
        
        # renderiza o quadrado usando toda a textura
        self.squareRenderer.render()
    
        # habilita o blending
        gl.glEnable(gl.GL_BLEND)
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(1.8, -1.05, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # atualiza a GPU com as coordenadas de textura do frame
        self.squareRenderer.updateVertexTextureCoord(self.fireTexCoordCurrentFrame)
        
        # renderiza o quadrado com parte da textura
        self.squareRenderer.render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.shaderProgram.releaseTexture2D()
        
        # desabilita o blending
        gl.glDisable(gl.GL_BLEND)

    def resizeGL(self, width, height):
        gl.glViewport(0, 0, width, height)
        
        # configura a projeção
        aspectRatio = width / height
        self.perspectiveMatrix = glm.perspective(glm.radians(60.0), aspectRatio, 0.1, 50.0)

def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(1200, 800)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

trying to open ./cg/images/textures/coins.jpg
opened file: size= (512, 256) format= JPEG mode= RGB
trying to open ./cg/images/textures/megaman_kindpng.png
opened file: size= (2705, 1028) format= PNG mode= RGBA
trying to open ./cg/images/textures/explosion_pngfuel.com.png
opened file: size= (1920, 1440) format= PNG mode= RGBA
trying to open ./cg/images/textures/fire_opengameart.org.png
opened file: size= (640, 384) format= PNG mode= RGBA


SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [20]:
! jupyter nbconvert --to python 39_Renderizando_sprites_animados.ipynb
%run -i 39_Renderizando_sprites_animados.py

[NbConvertApp] Converting notebook 37_Quadrado_com_texturas.ipynb to python
[NbConvertApp] Writing 8073 bytes to 37_Quadrado_com_texturas.py


trying to open ./pngwing.com_1.png
opened file: size= (650, 650) format= None mode= RGBA
trying to open ./pngwing.com_2.png
opened file: size= (1000, 1000) format= None mode= RGBA
trying to open ./pngwing.com_4.png
opened file: size= (650, 650) format= None mode= RGBA
